In [ ]:
import openai
import os

from dotenv import find_dotenv, load_dotenv


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

MODEL_NAME = os.environ["MODEL_NAME"]
MODEL_PROVIDER = os.environ["MODEL_PROVIDER"]

In [15]:
import nest_asyncio

from langsmith import traceable
from openai import OpenAI
from typing import Dict, List
from utils import get_vector_db_retriever


RAG_SYSTEM_PROMPT = """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)   # note: this is a langchain vector database retriever, so this .invoke() call will be traced automatically


@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)


@traceable
def call_openai(
    messages: List[Dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [16]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
ai_answer

"To trace with the @traceable decorator in Python, simply decorate any function you want to log traces for with @traceable. Ensure that the LANGSMITH_TRACING environment variable is set to 'true' and the LANGSMITH_API_KEY is configured with your API key. This allows you to log traces to LangSmith with minimal changes to your existing code."

In [ ]:
from langsmith import traceable

@traceable(
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)


@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    metadata={
        "model_name": MODEL_NAME, 
        "model_provider": MODEL_PROVIDER
    }
)
def call_openai(
    messages: List[Dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [18]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
ai_answer

'To add metadata to a run with @traceable, you can use the appropriate syntax in your code to specify key-value pairs that represent the metadata you want to attach. For detailed instructions, refer to the documentation on adding metadata to your traces. If you need further assistance, please consult the Concepts page for more information.'

In [19]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
ai_answer

'To add metadata at runtime, you can pass it in using the `langsmith_extra` parameter along with a unique `run_id`. For example, you can use the following code snippet: \n\n```python\nrun_id = str(uuid.uuid4())\nrag("where did harrison work", langsmith_extra={"run_id": run_id, "metadata": {"user_id": "harrison"}})\n```\n\nThis allows you to log information like a User ID dynamically during the execution of your function.'